In [1]:
%load_ext autoreload
%autoreload 2

import pickle
import matplotlib.pyplot as plt

In [2]:
from utils.train_medical import run_mimic_experiments, run_eeg_experiments, run_medical_experiments

# MIMIC-III: forecasting white blood cell count

In [ ]:
mimic_results = run_medical_experiments(retrain=True, dataset='mimic')

Training CPRNN
Epoch: 0	Train loss: 156.74915640694755
Epoch: 50	Train loss: 53.13235078539167
Epoch: 100	Train loss: 53.64533070155552
Epoch: 150	Train loss: 51.56313732692173
Epoch: 200	Train loss: 55.60751519884382
Epoch: 250	Train loss: 51.39998326982771
Epoch: 300	Train loss: 50.69910785130092
Epoch: 350	Train loss: 51.46189144679478
Epoch: 400	Train loss: 51.4951046534947
Epoch: 450	Train loss: 51.346938133239746
Epoch: 500	Train loss: 52.24199744633266
Epoch: 550	Train loss: 52.51100335802351
Epoch: 600	Train loss: 50.9170960017613
Epoch: 650	Train loss: 55.39422171456473
Epoch: 700	Train loss: 67.61379609789167
Epoch: 750	Train loss: 50.58711297171457
Epoch: 800	Train loss: 51.071151869637625
Epoch: 850	Train loss: 50.609366689409526
Epoch: 900	Train loss: 51.22157260349819
Epoch: 950	Train loss: 51.123785155160085
Training QRNN
Epoch:  0 | train loss: 2.4016
Epoch:  1 | train loss: 1.7792


# EEG dataset: forecasting EEG signals
Signals come from different distributions. Patients are shown different combinations of stimuli, and the recordings are taken from different parts of the brain.

Split 896 training instances to 600 / 296 true training/calibration.

(if using control data only, then 448 = 300 + 148)

Downsample training sequences to 50 steps
* 40 steps input, 10 prediction

In [ ]:
eeg_results = run_medical_experiments(retrain=True, dataset='eeg')